# Overview

This notebook is used for the development of the new orientation class that will function in place of FZfile.py and RotRep.

In [1]:
# import the functions that need to be migrated to the new classes

import os
import sys
import numpy as np

from tqdm import tqdm_notebook
from scipy.linalg import polar

sys.path.append("../../")

In [2]:
from RotRep import Mat2EulerZXZ, Mat2EulerZXZVectorized 
from RotRep import EulerZXZ2Mat, EulerZXZ2MatVectorized

from RotRep import Misorien2FZ1

from RotRep import rod_from_quaternion
from RotRep import Orien2FZ
from RotRep import GetSymRotMat

# from FZfile import generate_random_rot_mat
# from FZfile import misorien

In [3]:
from hexomap.orientation import Eulers
from hexomap.orientation import Quaternion
from hexomap.utility     import iszero
from hexomap.utility     import isone

The FZfile is somehow tied to pycuda, we have to come back for it later.

# Euler angles and (active) rotation matrix conversion

The previous implementation is done directly between the two, now we need to add quaternion to the fold.

In [3]:
EulerZXZ2Mat??

In [4]:
euler = [-1.26124505, -1.34263514, 0.39034601]
print(EulerZXZ2Mat(euler))

[[ 0.36369141  0.08331575  0.9277861 ]
 [-0.85460473  0.42614345  0.29673643]
 [-0.37064715 -0.90081088  0.22618675]]


In [5]:
from hexomap.orientation import Eulers

In [6]:
e = Eulers(*euler)
print(e)
print(e.as_matrix)

Eulers(phi1=-1.26124505, phi=-1.34263514, phi2=0.39034601, in_radians=True, order='zxz', convention='Bunge')
[[ 0.36369141  0.08331575  0.9277861 ]
 [-0.85460473  0.42614345  0.29673643]
 [-0.37064715 -0.90081088  0.22618675]]


In [7]:
# quick test

eulers = ((np.random.random(100*3)-0.5)*2*np.pi).reshape(100, 3)

for me in eulers:
    try:
        np.testing.assert_allclose(EulerZXZ2Mat(me), Eulers(*me).as_matrix)
    except:
        print("error")
        print(me)
        print(EulerZXZ2Mat(me))
        print(Eulers(*me).as_matrix)


In [8]:
EulerZXZ2MatVectorized??

In [12]:
N = 10000

eulers = ((np.random.random(N*3)-0.5)*2*np.pi).reshape(N, 3)

In [13]:
%%timeit

m_old = EulerZXZ2MatVectorized(eulers)

1.17 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%%timeit

m_new = Eulers.eulers_to_matrices(eulers)

1.2 ms ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
np.testing.assert_allclose(m_old, m_new)

In [12]:
Mat2EulerZXZ??

In [4]:
# original method
# euler -> matrix -> euler

euler = np.radians([0, 0, 32])  # special case, also need to make sure that the single rotation is the last one.
m = EulerZXZ2Mat(euler)

np.testing.assert_allclose(euler, Mat2EulerZXZ(m))

In [5]:
# new method

euler = Eulers(*np.radians([0, 0, 32]))

m = euler.as_matrix

print(Eulers.from_matrix(m))
print(euler)

Eulers(phi1=0.0, phi=0.0, phi2=0.5585053606381855, in_radians=True, order='zxz', convention='Bunge')
Eulers(phi1=0.0, phi=0.0, phi2=0.5585053606381855, in_radians=True, order='zxz', convention='Bunge')


Both methods are self consistent, now need to make sure that both can get the same Euler angles from a random rotation matrix

In [13]:
F = np.random.random(3*3).reshape(3,3)

R, V = polar(F)

print(R)
print(V)

[[ 0.1825966   0.97572875  0.12087964]
 [ 0.75865109 -0.21803073  0.61393088]
 [-0.62538548  0.02039622  0.78004935]]
[[0.15624347 0.26898015 0.30354132]
 [0.26898015 0.82666703 0.24860005]
 [0.30354132 0.24860005 1.2425504 ]]


In [14]:
euler_o = Mat2EulerZXZ(R)

euler_n = Eulers.from_matrix(R).as_array

print(np.array(euler_o)%(2*np.pi), euler_n)

[2.94718489 0.67605164 4.74499126] [2.94718489 0.67605164 4.74499126]


In [4]:
Mat2EulerZXZVectorized??

In [6]:
Fs = np.random.random(5*3*3).reshape(5,3,3)
Rs = np.array([polar(me)[0] for me in Fs])

eulers_o = Mat2EulerZXZVectorized(Rs)

print(eulers_o)

[[1.58458428 2.13042316 1.14246718]
 [2.02289707 1.9894906  1.22023214]
 [3.3984074  0.68974633 2.49756028]
 [4.0690013  0.78992427 6.1013702 ]
 [2.35568927 1.71723111 0.35105617]]


In [7]:
eulers_n = Eulers.matrices_to_eulers(Rs)

print(eulers_n)

[[1.58458428 2.13042316 1.14246718]
 [2.02289707 1.9894906  1.22023214]
 [3.3984074  0.68974633 2.49756028]
 [4.0690013  0.78992427 6.1013702 ]
 [2.35568927 1.71723111 0.35105617]]


In [8]:
# now batch testing

N = 10000
Fs = np.random.random(N*3*3).reshape(N,3,3)
Rs = np.array([polar(me)[0] for me in Fs])

In [9]:
%%timeit
eulers_o = Mat2EulerZXZVectorized(Rs)

2.01 ms ± 87.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
eulers_n = Eulers.matrices_to_eulers(Rs)

1.45 ms ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
np.testing.assert_allclose(eulers_o, eulers_n)

# Quaternion dev

In [4]:
euler = np.radians([33, 10, 1])

e = Eulers(*euler)
q = Quaternion.from_eulers(e)

print(np.degrees(q.as_eulers.as_array))

[33. 10.  1.]


In [7]:
for _ in tqdm_notebook(range(10000)):
    euler = Eulers(*((np.random.random(3)-0.5)*2*np.pi))
    q = Quaternion.from_eulers(euler)
    np.testing.assert_allclose(euler.as_array, q.as_eulers.as_array)

In [22]:
P = -1

def eu2qu(eu):
    """Euler angles to quaternion"""
    ee = 0.5*eu
    cPhi = np.cos(ee[1])
    sPhi = np.sin(ee[1])
    qu = np.array([ cPhi*np.cos(ee[0]+ee[2]),
                   -P*sPhi*np.cos(ee[0]-ee[2]),
                   -P*sPhi*np.sin(ee[0]-ee[2]),
                   -P*cPhi*np.sin(ee[0]+ee[2]) ])
    #if qu[0] < 0.0: qu.homomorph()                                                                   !ToDo: Check with original
    return qu

def qu2eu(qu):
    """Quaternion to Euler angles""" 
    q03 = qu[0]**2+qu[3]**2
    q12 = qu[1]**2+qu[2]**2
    chi = np.sqrt(q03*q12)
    
    if iszero(chi):
        eu = np.array([np.arctan2(-P*2.0*qu[0]*qu[3],qu[0]**2-qu[3]**2), 0.0,   0.0]) if iszero(q12) else \
        np.array([np.arctan2(2.0*qu[1]*qu[2],qu[1]**2-qu[2]**2),         np.pi, 0.0])
    else:
        eu = np.array([np.arctan2((-P*qu[0]*qu[2]+qu[1]*qu[3])*chi, (-P*qu[0]*qu[1]-qu[2]*qu[3])*chi ),
                       np.arctan2( 2.0*chi, q03-q12 ), 
                       np.arctan2(( P*qu[0]*qu[2]+qu[1]*qu[3])*chi, (-P*qu[0]*qu[1]+qu[2]*qu[3])*chi )])
    
    # reduce Euler angles to definition range, i.e a lower limit of 0.0
    eu = np.where(eu<0, (eu+2.0*np.pi)%np.array([2.0*np.pi,np.pi,2.0*np.pi]),eu)
    return eu

for _ in tqdm_notebook(range(10000)):
    eu = (np.random.random(3)-0.5)*np.pi*2
    eu = np.where(eu<0, (eu+2.0*np.pi)%np.array([2.0*np.pi,np.pi,2.0*np.pi]),eu%(2*np.pi))
    q = eu2qu(eu)
    np.testing.assert_allclose(eu, qu2eu(q))

In [15]:
F = np.random.random(3*3).reshape(3,3)

R, V = polar(F)

print(R)
print(np.degrees(Eulers.from_matrix(R).as_array))

q = Quaternion.from_matrix(R)

print(q.as_matrix)
print(np.degrees(q.as_eulers.as_array))

[[-0.25105867 -0.12553971  0.9597965 ]
 [ 0.93710801  0.21686823  0.27348992]
 [-0.24248322  0.96809501  0.06319762]]
[105.90467648  86.3766286  345.93816595]
[[-0.25105867 -0.12553971  0.9597965 ]
 [ 0.93710801  0.21686823  0.27348992]
 [-0.24248322  0.96809501  0.06319762]]
[105.90467648  86.3766286  345.93816595]


In [16]:
euler = Eulers(*((np.random.random(3)-0.5)*2*np.pi))

print(np.degrees(euler.as_array))

m = euler.as_matrix

print(m)

q = Quaternion.from_matrix(m)
print(np.degrees(q.as_eulers.as_array))

[ 68.16890986 331.94189004 352.08259557]
[[ 0.48116562 -0.76015152 -0.43663407]
 [ 0.87423211  0.45290461  0.17491606]
 [ 0.06479088 -0.46588312  0.882471  ]]
[248.16890986  28.05810996 172.08259557]


# Misorientation related calculation

In [15]:
Misorien2FZ1??